In [ ]:
%matplotlib inline
import holoviews as hv
hv.notebook_extension()
import sympy.interactive
sympy.interactive.init_printing('mathjax')
# import sys
# sys.path.append('/home/bnijholt/orbitalfield/')

In [ ]:
import kwant
from types import SimpleNamespace
import sympy
from sympy.physics.quantum import TensorProduct as kr
import numpy as np
from scipy.constants import hbar, m_e, eV, physical_constants
from discretizer import Discretizer, momentum_operators
import scipy.optimize
from itertools import product
k_B = physical_constants['Boltzmann constant in eV/K'][0] * 1000
sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
s0 = sympy.eye(2)
s0 = sympy.eye(2)
s0sz = np.kron(s0, sz)
s0s0 = np.kron(s0, s0)

# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental constants, in a SimpleNamespace.
constants = SimpleNamespace(
    m=0.015 * m_e,  # effective mass in kg
    a=10,  # lattice spacing in nm
    g=50,  # Lande factor
    hbar=hbar,
    m_e=m_e,
    e=eV,
    eV=eV,
    meV=eV * 1e-3)

constants.t = (hbar ** 2 / (2 * constants.m)) * (1e18 / constants.meV)  # meV * nm^2
constants.mu_B = physical_constants['Bohr magneton'][0] / constants.meV


def make_params(alpha=20,
                B_x=0,
                B_y=0,
                B_z=0,
                Delta=0.25,
                mu=0,
                orbital=True,
                t=constants.t,
                g=constants.g,
                mu_B=constants.mu_B,
                V=lambda x,y,z: 0,
                **kwargs):
    """Function that creates a namespace with parameters.

    Parameters:
    -----------
    alpha : float
        Spin-orbit coupling strength in units of meV*nm.
    B_x, B_y, B_z : float
        The magnetic field strength in the x, y and z direction in units of Tesla.
    Delta : float
        The superconducting gap in units of meV.
    mu : float
        The chemical potential in units of meV.
    orbital : bool
        Switches the orbital effects on and off.
    t : float
        Hopping parameter in meV * nm^2.
    g : float
        Lande g factor.
    mu_B : float
        Bohr magneton in meV/K.
    V : function
        Function of spatial coordinates (x, y, z) with is added to mu.

    Returns:
    --------
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    """
    p = SimpleNamespace(t=t,
                        g=g,
                        mu_B=mu_B,
                        alpha=alpha,
                        B_x=B_x,
                        B_y=B_y,
                        B_z=B_z,
                        Delta=Delta,
                        mu=mu,
                        orbital=orbital,
                        V=V,
                        **kwargs)
    return p


def discretized_hamiltonian(a=10):
    k_x, k_y, k_z = momentum_operators
    t, B_x, B_y, B_z, mu_B, Delta, mu, alpha, g, V = sympy.symbols('t B_x B_y B_z mu_B Delta mu alpha g V', real=True)
    t_interface = sympy.symbols('t_interface', real=True)
    k = sympy.sqrt(k_x**2+k_y**2+k_z**2)
    
    ham = ((t * k**2 - mu - V) * kr(s0, sz) +
               alpha * (k_y * kr(sx, sz) - k_x * kr(sy, sz)) +
               0.5 * g * mu_B * (B_x * kr(sx, s0) + B_y * kr(sy, s0) + B_z * kr(sz, s0)) +
               Delta * kr(s0, sx))
        
    args = dict(space_dependent={'V'}, lattice_constant=a, discrete_coordinates={'x', 'y', 'z'})

    tb_normal = Discretizer(ham.subs(Delta, 0), **args)
    tb_sc = Discretizer(ham.subs(g, 0), **args)
    tb_interface = Discretizer(ham.subs(t, t_interface), **args)
    return tb_normal, tb_sc, tb_interface


def matsubara_frequency(T, n):
    return (2*n + 1) * np.pi * T * 1j


def null_H(syst, p, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return np.linalg.inv(gf.data[::2, ::2])


def gf_from_H_0(H_0, t):
    H = np.copy(H_0)
    dim = t.shape[0]
    H[:dim, dim:] -= t.T.conj()
    H[dim:, :dim] -= t
    return np.linalg.inv(H)


def current_from_H_0(T, H_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    I = 0
    for H_0 in H_0s:
        gf = gf_from_H_0(H_0, t - H12)
        dim = t.shape[0]
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I


def null_Hs(syst, p, T, n, B):
    (p.B_x, p.B_y, p.B_z) = B
    return null_H(syst, p, T, n)


def I_c(syst, p, T, H_0s):
    """
    Example:
    --------
    I_cs = [I_c(syst, p, T, H_0s_) for p.B_x, H_0s_ in zip(Bs, H_0s)]
    """
    H12 = hopping(syst, [p])
    return scipy.optimize.minimize_scalar(lambda phase: -current_from_H_0(T, H_0s, H12, phase), 
                                          bounds=(-np.pi, np.pi), method='bounded')


T = 60e-3
matsfreqs = 20

# 3D system with Discretizer Hamiltonian, squared system

In [ ]:
def make_test_system(X, Y, Z, a=10):
    tb_normal, tb_sc = discretized_hamiltonian(a)[:-1]
    lat = tb_normal.lattice
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = tb_normal.onsite

    for hop, val in tb_normal.hoppings.items():
        syst[hop] = val

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = tb_sc.onsite

    for hop, val in tb_sc.hoppings.items():
        lead[hop] = val

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

T = 60e-3
matsfreqs = 20

syst, hopping = make_test_system(X=4, Y=5, Z=5)
p = make_params(Delta=0.25, alpha=20, mu=15)

H12 = hopping(syst, [p])
phases = np.linspace(-np.pi, np.pi, 51)
H_0s = [null_H(syst, p, T, n) for n in range(matsfreqs)]
I = np.array([current_from_H_0(T, H_0s, H12, phase) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# Simpler way (but slightly slower)

In [ ]:
def null_G(syst, p, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return gf.data[::2, ::2]

def current_from_G_0(T, G_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

G_0s = [null_G(syst, p, T, n) for n in range(matsfreqs)]
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(T, G_0s, H12, phase) for phase in phases]
I = np.array(I)
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# 3D test system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3))
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    range_Z = range(Z)
    syst[(lat(x, y, z) for x in range_X for y in range_Y for z in range_Z)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range_Y for z in range_Z)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


In [ ]:
delta = 0.01
T = 0.5 * delta
matsfreqs = 200
syst, hopping = make_test_system(X=4, Y=5, Z=5)
p = SimpleNamespace(mu=0., t=1., delta=1)

H12 = hopping(syst, [p])
phases = np.linspace(-np.pi, np.pi, 51)
H_0s = [null_H(syst, p, T, n) for n in range(matsfreqs)]
I = np.array([current_from_H_0(T, H_0s, H12, phase) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

In [ ]:
kwant.plot(syst)